In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import yfinance as yf
import sys
sys.path.append('../data_func')
from data_helper_functions import create_study_periods,create_tensors

In [3]:
df=pd.read_csv('../data/crsp_ff_adjusted.csv')
df['RET']=pd.to_numeric(df['RET'],errors='coerce')
df['date'] = pd.to_datetime(df['date'])
df.dropna(subset=['RET'],inplace=True)
#drop unamed 0 column
df.drop(columns=['Unnamed: 0'],inplace=True)

In [6]:
#select returns to use
returns='Adj_RET_Mkt'
df=df[['date','TICKER',f'{returns}']]
if returns!='RET':
    #rename returns column
    df.rename(columns={f'{returns}':'RET'},inplace=True)

/Users/jacobbarcelona/.pyenv/versions/3.9.16/lib/python3.9/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [25]:
#Optional parameter target_type: 'cross_sectional_median(default)','buckets(10 buckets)','raw_returns'.
study_periods=create_study_periods(df,n_periods=23,window_size=240,trade_size=250,train_size=750,forward_roll=250,start_date=datetime(1990,1,1),end_date=datetime(2015,12,31))

 86%|████████▌ | 25/29 [00:10<00:01,  2.42it/s]

Reached the end of the dataset.


In [29]:
train_test_splits=create_tensors(study_periods)


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:  3.4min
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:  3.6min
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:  9.7min
[Parallel(n_jobs=6)]: Done  17 out of  25 | elapsed:  9.9min remaining:  4.7min
[Parallel(n_jobs=6)]: Done  20 out of  25 | elapsed: 12.9min remaining:  3.2min
[Parallel(n_jobs=6)]: Done  23 out of  25 | elapsed: 13.1min remaining:  1.1min
[Parallel(n_jobs=6)]: Done  25 out of  25 | elapsed: 14.6min finished


In [9]:
#Optional code to verify tensor shapes
# for train_data, train_labels, test_data, test_labels in train_test_splits:
#     print(train_data.shape, train_labels.shape, test_data.shape, test_labels.shape)

torch.Size([259482, 240, 1]) torch.Size([259482]) torch.Size([86423, 240, 1]) torch.Size([86423])
torch.Size([259523, 240, 1]) torch.Size([259523]) torch.Size([85610, 240, 1]) torch.Size([85610])
torch.Size([257790, 240, 1]) torch.Size([257790]) torch.Size([86667, 240, 1]) torch.Size([86667])
torch.Size([259178, 240, 1]) torch.Size([259178]) torch.Size([86360, 240, 1]) torch.Size([86360])
torch.Size([259115, 240, 1]) torch.Size([259115]) torch.Size([85279, 240, 1]) torch.Size([85279])
torch.Size([258784, 240, 1]) torch.Size([258784]) torch.Size([84181, 240, 1]) torch.Size([84181])
torch.Size([256298, 240, 1]) torch.Size([256298]) torch.Size([86916, 240, 1]) torch.Size([86916])
torch.Size([256854, 240, 1]) torch.Size([256854]) torch.Size([85044, 240, 1]) torch.Size([85044])
torch.Size([256619, 240, 1]) torch.Size([256619]) torch.Size([86592, 240, 1]) torch.Size([86592])
torch.Size([259030, 240, 1]) torch.Size([259030]) torch.Size([86542, 240, 1]) torch.Size([86542])
torch.Size([258656, 

Right now our prediction task is predicting a stocks returns above or below the cross section rolling median of returns. But I want to be able to reject making a prediction, (a -1 choice) when I am not confident in the prediction.
We can train a model g(x) that rejects or acepts data to make a prediction and a model f(x) that predicts the returns. We can then combine the two models to make a prediction.

If I can somehow make a stochastic transformer, and during testing reject data points that are too far from our self-attention centroids

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Checking if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Define the LSTM Classifier model
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True, dropout=0.1)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])  # We only want the last output of the sequence
        return out

model = LSTMClassifier(input_size=1, hidden_size=25, output_size=2).to(device)  # Move model to GPU if available
# Loss depends on target, MAE for returns, Cross Entropy for above/below cross-sectional median. Also have selective loss in utils
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.001)

# Parameters
patience = 10
n_epochs = 1000
best_val_loss = float('inf')
counter = 0

# Training Loop
for epoch in range(n_epochs):
    train_loss = 0.0
    val_loss = 0.0
    
    for train_data, train_labels, val_data, val_labels in tqdm(train_test_splits):
        model.train()

        train_data, train_labels = train_data.to(device), train_labels.to(device)  # Move data to GPU if available

        # Convert the dataset into DataLoader for batching
        train_dataset = TensorDataset(train_data, train_labels)
        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

        for sequences, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Validation step
        model.eval()

        val_data, val_labels = val_data.to(device), val_labels.to(device)  # Move data to GPU if available
        
        # Adding DataLoader for validation data
        val_dataset = TensorDataset(val_data, val_labels)
        val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
        
        with torch.no_grad():
            for sequences, labels in val_loader:
                outputs = model(sequences)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

    train_loss /= len(train_loader)
    val_loss /= len(val_loader)

    print(f"Epoch {epoch+1}, Train Loss: {train_loss}, Validation Loss: {val_loss}")

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1

    if counter == patience:
        print("Early stopping triggered")
        break


In [ ]:
model.load_state_dict(best_model_state)
model.eval()

in_sample_long_portfolios = pd.DataFrame()
out_of_sample_long_portfolios = pd.DataFrame()

in_sample_short_portfolios = pd.DataFrame()
out_of_sample_short_portfolios = pd.DataFrame()

k = 10  # Number of top assets to select in portfolios

for train_data, train_labels, val_data, val_labels in tqdm(train_test_splits):
    
    with torch.no_grad():
        train_predictions = model(train_data.to(device)
        val_predictions = model(val_data.to(device))

        train_probs = torch.softmax(train_predictions, dim=1)[:, 1].cpu().numpy()
        val_probs = torch.softmax(val_predictions, dim=1)[:, 1].cpu().numpy()

    # Assuming you have a dataframe or similar structure to hold the date and TICKER information
    train_df['predicted_prob'] = train_probs
    val_df['predicted_prob'] = val_probs

    # In-Sample Portfolio Construction
    for date in train_df['date'].unique():
        date_data = train_df[train_df['date'] == date].sort_values(by='predicted_prob', ascending=False)
        
        long_tickers = date_data.head(k)
        short_tickers = date_data.tail(k)
        
        in_sample_long_portfolios = pd.concat([in_sample_long_portfolios, long_tickers])
        in_sample_short_portfolios = pd.concat([in_sample_short_portfolios, short_tickers])

    # Out-of-Sample Portfolio Construction
    for date in val_df['date'].unique():
        date_data = val_df[val_df['date'] == date].sort_values(by='predicted_prob', ascending=False)
        
        long_tickers = date_data.head(k)
        short_tickers = date_data.tail(k)
        
        out_of_sample_long_portfolios = pd.concat([out_of_sample_long_portfolios, long_tickers])
        out_of_sample_short_portfolios = pd.concat([out_of_sample_short_portfolios, short_tickers])





In [ ]:
in_sample_long_portfolios.to_csv('../data/ltsm_results/in_sample_long_portfolios.csv')
in_sample_short_portfolios.to_csv('../data/ltsm_results/in_sample_short_portfolios.csv')
out_of_sample_long_portfolios.to_csv('../data/ltsm_results/out_of_sample_long_portfolios.csv')
out_of_sample_short_portfolios.to_csv('../data/ltsm_results/out_of_sample_short_portfolios.csv')